# VTK File Processing Notebook

This notebook automates the workflow of:
1. Running an active program 
2. Processing .vtk files from a specified folder
3. Saving the processed data appropriately

## Requirements
- PyVista (for VTK file processing)
- NumPy (for numerical operations)
- Matplotlib (for visualization, if needed)

In [1]:
# Install required libraries if not already installed
import sys
import subprocess

# Install required packages if not already installed
try:
    import pyvista
    import numpy as np
except ImportError:
    print("Installing required packages...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyvista", "numpy", "matplotlib"])
    import pyvista
    import numpy as np
    import matplotlib.pyplot as plt

print("Required libraries loaded successfully!")

Required libraries loaded successfully!


## 1. Run the Active Program

This section uses the `subprocess` module to execute an external program and capture its output. You can modify the program path and arguments as needed.

In [2]:
import subprocess
import os
import time

def run_active_program(program_path, args=None):
    """
    Run an external program and capture its output
    
    Parameters:
    -----------
    program_path : str
        Path to the executable
    args : list, optional
        Command line arguments for the program
        
    Returns:
    --------
    tuple
        (return_code, stdout, stderr)
    """
    if args is None:
        args = []
    
    try:
        # Create the full command
        cmd = [program_path] + args
        
        # Print the command being executed
        print(f"Executing: {' '.join(cmd)}")
        
        # Run the program and capture output
        start_time = time.time()
        process = subprocess.Popen(
            cmd, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        
        # Read output in real-time
        stdout, stderr = process.communicate()
        
        end_time = time.time()
        
        # Check the return code
        if process.returncode == 0:
            print(f"Program executed successfully in {end_time - start_time:.2f} seconds")
        else:
            print(f"Program execution failed with return code {process.returncode}")
            
        return process.returncode, stdout, stderr
    
    except Exception as e:
        print(f"Error running program: {e}")
        return -1, "", str(e)

# Example usage (comment out or modify as needed)
# program_path = "/path/to/your/program"
# args = ["--param1", "value1", "--param2", "value2"]
# return_code, stdout, stderr = run_active_program(program_path, args)

# print("\nStandard Output:")
# print(stdout)

# if stderr:
#     print("\nStandard Error:")
#     print(stderr)

## 2. Specify Folder Path

This section allows you to specify the folder containing the .vtk files for processing. You can:
- Enter the path directly
- Choose from common directories 
- Browse using a file dialog (when running in an interactive environment)

In [3]:
import os
import glob
from IPython.display import display, clear_output
import ipywidgets as widgets

# Function to get VTK files from a directory
def get_vtk_files(directory):
    """
    Get all .vtk files in the specified directory
    
    Parameters:
    -----------
    directory : str
        Directory path to search for VTK files
        
    Returns:
    --------
    list
        List of full paths to VTK files
    """
    vtk_pattern = os.path.join(directory, "*.vtk")
    files = glob.glob(vtk_pattern)
    return sorted(files)

# Create a text input for manual path entry
manual_path_input = widgets.Text(
    value='',
    placeholder='Enter directory path containing .vtk files',
    description='Path:',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

# Create a dropdown for common directories
common_paths = [
    os.path.abspath(os.path.expanduser("~/Downloads")),
    os.path.abspath(os.path.expanduser("~/Documents")),
    os.path.abspath(os.getcwd()),
    "/home/abdua786/code/uni/3/dissertation/dissertation/data"  # Example project-specific path
]

common_paths_dropdown = widgets.Dropdown(
    options=[('Select common location...', '')] + [(os.path.basename(p), p) for p in common_paths],
    value='',
    description='Common locations:',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

# Function to update the manual path when a common path is selected
def on_common_path_change(change):
    if change['type'] == 'change' and change['name'] == 'value' and change['new']:
        manual_path_input.value = change['new']

common_paths_dropdown.observe(on_common_path_change)

# Create a button to check for VTK files
check_files_button = widgets.Button(
    description='Check for VTK files',
    disabled=False,
    button_style='info', 
    tooltip='Check the directory for VTK files',
    icon='search'
)

output_area = widgets.Output()

# Function to check for VTK files when the button is clicked
def on_check_files_button_clicked(b):
    with output_area:
        clear_output()
        
        directory = manual_path_input.value.strip()
        if not directory:
            print("Please enter a directory path")
            return
            
        if not os.path.isdir(directory):
            print(f"Error: '{directory}' is not a valid directory")
            return
            
        vtk_files = get_vtk_files(directory)
        
        if not vtk_files:
            print(f"No VTK files found in {directory}")
        else:
            print(f"Found {len(vtk_files)} VTK files in {directory}:")
            for i, file in enumerate(vtk_files, 1):
                print(f"{i}. {os.path.basename(file)}")
            
            global selected_directory
            selected_directory = directory

check_files_button.on_click(on_check_files_button_clicked)

# Optional file browser for interactive environments
try:
    # Only import if in a Jupyter environment
    from ipyfilechooser import FileChooser
    
    file_chooser = FileChooser(
        path=os.getcwd(),
        filename='',
        title='Select Directory:',
        show_hidden=False,
        select_default=True,
        directory_select=True
    )
    
    file_chooser_button = widgets.Button(
        description='Browse...',
        disabled=False,
        button_style='', 
        tooltip='Browse for directory',
        icon='folder-open'
    )
    
    def on_file_chooser_button_clicked(b):
        display(file_chooser)
        
    file_chooser_button.on_click(on_file_chooser_button_clicked)
    
    def on_file_chooser_change(chooser):
        manual_path_input.value = chooser.selected
        
    file_chooser.register_callback(on_file_chooser_change)
    
    # Display all widgets
    display(widgets.HBox([manual_path_input, file_chooser_button]))
    display(common_paths_dropdown)
    display(check_files_button)
    display(output_area)
    
except ImportError:
    # Fallback for non-interactive environments
    print("Running in non-interactive environment. File browser not available.")
    # Define a variable to store the selected directory
    selected_directory = ''
    
    # Display available widgets
    display(manual_path_input)
    display(common_paths_dropdown)
    display(check_files_button)
    display(output_area)
    
    # Function to manually set the directory
    def set_directory(path):
        global selected_directory
        manual_path_input.value = path
        selected_directory = path
        vtk_files = get_vtk_files(path)
        print(f"Found {len(vtk_files)} VTK files in {path}")

Running in non-interactive environment. File browser not available.


Text(value='', description='Path:', layout=Layout(width='80%'), placeholder='Enter directory path containing .…

Dropdown(description='Common locations:', layout=Layout(width='80%'), options=(('Select common location...', '…

Button(button_style='info', description='Check for VTK files', icon='search', style=ButtonStyle(), tooltip='Ch…

Output()

## 3. Load and Process `.vtk` Files

This section uses PyVista to load each .vtk file from the specified folder and process it. You can customize the processing based on your specific needs.

In [ ]:
import pyvista as pv
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import time

# Function to process a single VTK file
def process_vtk_file(file_path, processing_options=None):
    """
    Load and process a single VTK file
    
    Parameters:
    -----------
    file_path : str
        Path to the VTK file
    processing_options : dict, optional
        Options for processing (e.g., filters, transformations)
        
    Returns:
    --------
    tuple
        (processed_data, metadata)
    """
    try:
        # Load the VTK file
        mesh = pv.read(file_path)
        
        # Extract basic metadata
        metadata = {
            'filename': os.path.basename(file_path),
            'n_points': mesh.n_points,
            'n_cells': mesh.n_cells,
            'bounds': mesh.bounds,
            'volume': mesh.volume if hasattr(mesh, 'volume') else None,
            'point_arrays': list(mesh.point_data.keys()),
            'cell_arrays': list(mesh.cell_data.keys()),
            'field_arrays': list(mesh.field_data.keys()) if hasattr(mesh, 'field_data') else []
        }
        
        # Apply processing based on options
        processed_data = mesh
        
        if processing_options:
            # Apply filters and transformations based on options
            if processing_options.get('smooth', False):
                processed_data = processed_data.smooth(n_iter=processing_options.get('smooth_iterations', 100))
                
            if processing_options.get('decimate', False):
                target_reduction = processing_options.get('decimate_target', 0.5)
                processed_data = processed_data.decimate(target_reduction)
                
            if processing_options.get('clip', False) and 'clip_normal' in processing_options:
                normal = processing_options.get('clip_normal')
                origin = processing_options.get('clip_origin', [0, 0, 0])
                processed_data = processed_data.clip(normal, origin)
        
        return processed_data, metadata
    
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None, {'filename': os.path.basename(file_path), 'error': str(e)}

# Function to process all VTK files in a directory
def process_all_vtk_files(directory, processing_options=None, max_files=None, show_progress=True):
    """
    Process all VTK files in a directory
    
    Parameters:
    -----------
    directory : str
        Directory containing VTK files
    processing_options : dict, optional
        Options for processing each file
    max_files : int, optional
        Maximum number of files to process
    show_progress : bool, optional
        Whether to show a progress bar
        
    Returns:
    --------
    tuple
        (processed_data_dict, metadata_list)
    """
    vtk_files = get_vtk_files(directory)
    
    if max_files is not None:
        vtk_files = vtk_files[:max_files]
    
    processed_data = {}
    metadata_list = []
    
    if show_progress:
        # Use tqdm for progress tracking
        iterator = tqdm(vtk_files, desc="Processing VTK files")
    else:
        iterator = vtk_files
    
    for file_path in iterator:
        result, metadata = process_vtk_file(file_path, processing_options)
        
        if result is not None:
            filename = os.path.basename(file_path)
            processed_data[filename] = result
            metadata_list.append(metadata)
    
    return processed_data, metadata_list

# Example for interactive processing - can be customized based on your needs
process_button = widgets.Button(
    description='Process VTK Files',
    disabled=False,
    button_style='success', 
    tooltip='Process all VTK files in the selected directory',
    icon='gears'
)

# Processing options UI elements
smooth_checkbox = widgets.Checkbox(
    value=False,
    description='Apply smoothing',
    disabled=False
)

smooth_iterations = widgets.IntSlider(
    value=100,
    min=1,
    max=1000,
    step=10,
    description='Iterations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

decimate_checkbox = widgets.Checkbox(
    value=False,
    description='Apply decimation',
    disabled=False
)

decimate_target = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=0.9,
    step=0.05,
    description='Target reduction:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

max_files_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Max files:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

processing_output = widgets.Output()

# Function to process files when the button is clicked
def on_process_button_clicked(b):
    with processing_output:
        clear_output()
        
        if not hasattr(sys.modules[__name__], 'selected_directory') or not selected_directory:
            print("Please select a directory first")
            return
        
        # Gather processing options
        processing_options = {
            'smooth': smooth_checkbox.value,
            'smooth_iterations': smooth_iterations.value,
            'decimate': decimate_checkbox.value,
            'decimate_target': decimate_target.value
        }
        
        print(f"Processing VTK files in {selected_directory} with options:")
        for key, value in processing_options.items():
            print(f"  - {key}: {value}")
        
        start_time = time.time()
        
        # Process the files
        global processed_data, metadata_list
        processed_data, metadata_list = process_all_vtk_files(
            selected_directory, 
            processing_options,
            max_files=max_files_slider.value
        )
        
        end_time = time.time()
        
        # Display results
        print(f"\nProcessed {len(processed_data)} files in {end_time - start_time:.2f} seconds")
        
        if metadata_list:
            print("\nMetadata summary:")
            df = pd.DataFrame(metadata_list)
            display(df)
            
            # Visualization example (if applicable)
            if len(processed_data) > 0:
                first_mesh = list(processed_data.values())[0]
                print("\nVisualization example (first mesh):")
                plotter = pv.Plotter(notebook=True)
                plotter.add_mesh(first_mesh, show_edges=True)
                plotter.show_grid()
                plotter.show()

process_button.on_click(on_process_button_clicked)

# Display processing controls
print("VTK Processing Options:")
display(widgets.VBox([
    widgets.HBox([smooth_checkbox, smooth_iterations]),
    widgets.HBox([decimate_checkbox, decimate_target]),
    max_files_slider,
    process_button
]))
display(processing_output)

## 4. Save Processed Files

This section saves the processed VTK data to appropriate formats. Options include:
- Saving as new VTK files
- Exporting mesh data to CSV/NumPy arrays
- Exporting metadata to JSON/CSV

In [ ]:
import json
import shutil
import datetime

# Function to create an output directory
def create_output_directory(base_dir=None, prefix="processed"):
    """
    Create an output directory for saving processed files
    
    Parameters:
    -----------
    base_dir : str, optional
        Base directory where the output folder will be created
    prefix : str, optional
        Prefix for the output folder name
        
    Returns:
    --------
    str
        Path to the created output directory
    """
    if base_dir is None:
        if hasattr(sys.modules[__name__], 'selected_directory') and selected_directory:
            base_dir = selected_directory
        else:
            base_dir = os.getcwd()
    
    # Create a timestamp-based folder name
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(base_dir, f"{prefix}_{timestamp}")
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    return output_dir

# Function to save a single processed mesh
def save_processed_mesh(mesh, output_path, file_format="vtk"):
    """
    Save a processed mesh to a file
    
    Parameters:
    -----------
    mesh : pyvista.DataSet
        The mesh to save
    output_path : str
        Base path for the output file (without extension)
    file_format : str, optional
        Format to save the file in ('vtk', 'vtp', 'stl', etc.)
        
    Returns:
    --------
    str
        Path to the saved file
    """
    # Add the extension if not present
    if not output_path.lower().endswith(f".{file_format.lower()}"):
        output_path = f"{output_path}.{file_format.lower()}"
    
    # Save the mesh
    mesh.save(output_path)
    
    return output_path

# Function to export mesh data to CSV
def export_mesh_to_csv(mesh, output_path):
    """
    Export mesh point data to CSV
    
    Parameters:
    -----------
    mesh : pyvista.DataSet
        The mesh to export
    output_path : str
        Path for the output CSV file
        
    Returns:
    --------
    str
        Path to the saved file
    """
    # Get points as a numpy array
    points = mesh.points
    
    # Create a DataFrame with the points
    df = pd.DataFrame(points, columns=['x', 'y', 'z'])
    
    # Add any point data arrays to the DataFrame
    for name, data in mesh.point_data.items():
        if data.ndim == 1:  # Only add 1D arrays
            df[name] = data
        elif data.ndim == 2 and data.shape[1] <= 3:  # For vector data
            for i, comp in enumerate(['x', 'y', 'z'][:data.shape[1]]):
                df[f"{name}_{comp}"] = data[:, i]
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    
    return output_path

# Function to save all processed meshes
def save_all_processed_data(processed_data, metadata_list, output_dir=None, 
                           formats=None, save_metadata=True):
    """
    Save all processed data to files
    
    Parameters:
    -----------
    processed_data : dict
        Dictionary of processed meshes
    metadata_list : list
        List of metadata dictionaries
    output_dir : str, optional
        Directory to save the files to
    formats : list, optional
        List of formats to save the meshes in
    save_metadata : bool, optional
        Whether to save the metadata to a JSON/CSV file
        
    Returns:
    --------
    dict
        Dictionary with information about saved files
    """
    if output_dir is None:
        output_dir = create_output_directory()
    
    if formats is None:
        formats = ["vtk"]  # Default format
    
    # Ensure formats is a list
    if isinstance(formats, str):
        formats = [formats]
    
    print(f"Saving processed data to {output_dir}")
    
    saved_files = {
        'meshes': {},
        'metadata': None,
        'csv_data': {}
    }
    
    # Save each processed mesh
    for filename, mesh in processed_data.items():
        base_name = os.path.splitext(filename)[0]
        mesh_saved_files = []
        
        for fmt in formats:
            output_path = os.path.join(output_dir, f"{base_name}.{fmt}")
            saved_path = save_processed_mesh(mesh, output_path, fmt)
            mesh_saved_files.append(saved_path)
        
        # Export to CSV if requested
        if 'csv' in formats:
            csv_path = os.path.join(output_dir, f"{base_name}_points.csv")
            saved_csv = export_mesh_to_csv(mesh, csv_path)
            saved_files['csv_data'][filename] = saved_csv
        
        saved_files['meshes'][filename] = mesh_saved_files
    
    # Save metadata
    if save_metadata and metadata_list:
        # Save as JSON
        metadata_json_path = os.path.join(output_dir, "metadata.json")
        with open(metadata_json_path, 'w') as f:
            json.dump(metadata_list, f, indent=2)
        
        # Save as CSV
        metadata_csv_path = os.path.join(output_dir, "metadata.csv")
        metadata_df = pd.DataFrame(metadata_list)
        metadata_df.to_csv(metadata_csv_path, index=False)
        
        saved_files['metadata'] = {
            'json': metadata_json_path,
            'csv': metadata_csv_path
        }
    
    # Create a README file with processing information
    readme_path = os.path.join(output_dir, "README.txt")
    with open(readme_path, 'w') as f:
        f.write(f"Processed VTK Files\n")
        f.write(f"=================\n\n")
        f.write(f"Processing date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(f"Number of files processed: {len(processed_data)}\n")
        f.write(f"Formats saved: {', '.join(formats)}\n\n")
        f.write(f"File listing:\n")
        
        for filename in processed_data.keys():
            f.write(f"- {filename}\n")
    
    print(f"Saved {len(processed_data)} files in {len(formats)} format(s)")
    print(f"Total files saved: {sum(len(files) for files in saved_files['meshes'].values())}")
    
    if save_metadata:
        print(f"Metadata saved to {metadata_json_path} and {metadata_csv_path}")
    
    return saved_files

# Create UI for saving files
save_button = widgets.Button(
    description='Save Processed Files',
    disabled=False,
    button_style='warning', 
    tooltip='Save processed files to disk',
    icon='save'
)

# Format selection checkboxes
format_vtk = widgets.Checkbox(value=True, description='VTK', disabled=False)
format_vtp = widgets.Checkbox(value=False, description='VTP', disabled=False)
format_stl = widgets.Checkbox(value=False, description='STL', disabled=False)
format_csv = widgets.Checkbox(value=True, description='CSV (points)', disabled=False)

# Custom output directory option
custom_output_dir = widgets.Text(
    value='',
    placeholder='Leave empty for auto-generated directory',
    description='Output directory:',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

save_metadata = widgets.Checkbox(
    value=True,
    description='Save metadata (JSON/CSV)',
    disabled=False
)

save_output = widgets.Output()

# Function to handle save button click
def on_save_button_clicked(b):
    with save_output:
        clear_output()
        
        # Check if we have processed data
        if not hasattr(sys.modules[__name__], 'processed_data') or not processed_data:
            print("No processed data available. Please process VTK files first.")
            return
        
        # Collect selected formats
        selected_formats = []
        if format_vtk.value:
            selected_formats.append("vtk")
        if format_vtp.value:
            selected_formats.append("vtp")
        if format_stl.value:
            selected_formats.append("stl")
        if format_csv.value:
            selected_formats.append("csv")
        
        if not selected_formats:
            print("Please select at least one output format")
            return
        
        # Determine output directory
        output_dir = custom_output_dir.value.strip()
        if not output_dir:
            output_dir = create_output_directory()
        else:
            os.makedirs(output_dir, exist_ok=True)
        
        # Save the files
        saved_files = save_all_processed_data(
            processed_data,
            metadata_list,
            output_dir=output_dir,
            formats=selected_formats,
            save_metadata=save_metadata.value
        )
        
        print(f"\nAll files saved successfully to: {output_dir}")
        
        # Display a sample of the saved files
        if saved_files['meshes']:
            example_file = list(saved_files['meshes'].keys())[0]
            example_paths = saved_files['meshes'][example_file]
            print(f"\nExample for '{example_file}':")
            for path in example_paths:
                print(f"  - {os.path.basename(path)}")
        
        if save_metadata.value and saved_files['metadata']:
            print(f"\nMetadata saved to:")
            for fmt, path in saved_files['metadata'].items():
                print(f"  - {os.path.basename(path)} ({fmt})")

save_button.on_click(on_save_button_clicked)

# Display save controls
print("Save Options:")
display(widgets.VBox([
    widgets.HBox([
        widgets.Label('Output formats:'), 
        format_vtk, format_vtp, format_stl, format_csv
    ]),
    custom_output_dir,
    save_metadata,
    save_button
]))
display(save_output)

## Conclusion

This notebook provides a workflow to:
1. Run an active program
2. Process VTK files from a specified folder
3. Apply various transformations to the meshes 
4. Save the processed data in multiple formats

### Next Steps

You can extend this notebook by:
- Adding more visualization options using PyVista's plotting capabilities
- Implementing custom processing algorithms specific to your data
- Adding batch processing for large datasets
- Connecting this workflow to other analysis tools